In [ ]:
import snowflake.connector
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import kaggle


In [ ]:
# Download the dataset
!kaggle datasets download -d hanaksoy/customer-purchasing-behaviors

# Unzip and load into Snowflake
import zipfile
with zipfile.ZipFile('customer-purchasing-behaviors.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


In [ ]:
# Snowflake connection details
def create_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='YOUR_USER',
            password='YOUR_PASSWORD',
            account='YOUR_ACCOUNT',
            role='YOUR_ROLE',
            warehouse='YOUR_WAREHOUSE',
            database='YOUR_DATABASE',
            schema='YOUR_SCHEMA'
        )
        return conn
    except snowflake.connector.errors.ProgrammingError as e:
        print(f"Programming error: {e}")
    except snowflake.connector.errors.DatabaseError as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")


In [ ]:
# Create a table in Snowflake
create_table_query = """
CREATE TABLE customer_purchasing_behavior (
    customer_id INT,
    age INT,
    location STRING,
    purchase_amount FLOAT,
    purchase_date DATE
)
"""
conn.cursor().execute(create_table_query)


In [ ]:
# Fetch data from Snowflake
def fetch_customer_data():
    conn = create_snowflake_connection()
    query = "SELECT * FROM customer_purchasing_behavior;"
    data = pd.read_sql(query, conn)
    conn.close()
    return data


In [ ]:
# Load and display data
data = fetch_customer_data()
data.head()

In [ ]:
# Streamlit Application
def main():
    st.title('Customer Purchasing Behavior Analysis')

    # Load data
    data = fetch_customer_data()

    # Filter data by customer segments
    age_group = st.selectbox('Select Age Group:', data['age'].unique())
    region = st.selectbox('Select Region:', data['location'].unique())
    filtered_data = data[(data['age'] == age_group) & (data['location'] == region)]

    st.subheader('Filtered Data')
    st.write(filtered_data)

    # Visualize purchase behavior over time
    st.subheader('Purchasing Trends')
    plt.figure(figsize=(10, 5))
    plt.plot(filtered_data['purchase_date'], filtered_data['purchase_amount'], label='Purchase Amount', marker='o')
    plt.title('Purchasing Trends')
    plt.xlabel('Purchase Date')
    plt.ylabel('Amount')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    st.pyplot(plt)

    # Display summary statistics
    st.subheader('Summary Statistics')
    total_purchases = filtered_data['purchase_amount'].sum()
    avg_purchase_amount = filtered_data['purchase_amount'].mean()
    customer_count = filtered_data['customer_id'].nunique()
    st.write(f'Total Purchases: {total_purchases}')
    st.write(f'Average Purchase Amount: {avg_purchase_amount}')
    st.write(f'Customer Count: {customer_count}')

if __name__ == '__main__':
    main()
